In [ ]:
import numpy as np
import limp

import pandas as pd
engines = pd.read_csv('Endless_Sky_engine_data.csv')

# Endless Sky - engine choices

## Weighted sum of thrust and turn
Per [original blog post](https://shapr.github.io/posts/2019-07-10-smt-solvers.html)

In [ ]:
p = limp.Problem()
obj = limp.Expr()
total_cost = limp.Expr()
max_cost = 210 # weight limit our ship can carry

for row in engines.itertuples():
    N_eng = p.intvar(row.name, 0, max_cost // row.weight)
    obj += (row.thrust + row.turn/36)*N_eng
    total_cost += row.weight * N_eng
p.constraint(total_cost, max_cost)
ans = p.maximize(obj)

for eng, cnt in ans['by_name'].items():
    if cnt > 0:
        print(f'{cnt:.0f}    {eng}')
x = ans['soln'].x
print(f'Total cost:   {np.dot(engines.weight, x)} of {max_cost}')
print(f'Total thrust: {np.dot(engines.thrust, x)}')
print(f'Total turn:   {np.dot(engines.turn, x)}')

## Vectorized version

In [ ]:
p = limp.Problem()
max_cost = 210 # weight limit our ship can carry

N_eng = [p.intvar(row.name, 0, max_cost // row.weight) for row in engines.itertuples()]
p.constraint( p.sum(N_eng, engines.weight), max_cost )
ans = p.maximize(   p.sum(N_eng, engines.thrust + engines.turn/36) ) # 1534 + 40050
# ans = p.maximize(   p.sum(N_eng, engines.thrust) ) # 2514
# ans = p.maximize(   p.sum(N_eng, engines.turn) ) # 90660
# max thurst vs max turn is indeed a 1:36 ratio...

for eng, cnt in ans['by_name'].items():
    if cnt > 0:
        print(f'{cnt:.0f}    {eng}')
x = ans['soln'].x
print(f'Total cost:   {np.dot(engines.weight, x)} of {max_cost}')
print(f'Total thrust: {np.dot(engines.thrust, x)}')
print(f'Total turn:   {np.dot(engines.turn, x)/36}')

## Balance thrust and turning with abs()

If we try to balance thrust and (scaled) turning, there are a range of solutions that achieve a perfect balance.
If we try to balance them and break ties by maximum thrust, there's a unique solution, but it takes longer to find.  The smaller the multiplier for total thrust, the longer the solution takes too:  1e-6 takes twice as long as 1e-3, for the same answer.  Weirdly, 1e-4 is MUCH slower than either one (3-6x).

In [ ]:
p = limp.Problem()
max_cost = 210 # weight limit our ship can carry
min_cost = 0.75 * max_cost

N_eng = [p.intvar(row.name, 0, max_cost // row.weight) for row in engines.itertuples()]
p.constraint( min_cost, p.sum(N_eng, engines.weight), max_cost )
tot_thrust = p.sum(N_eng, engines.thrust)
balance = p.sum(N_eng, engines.thrust - engines.turn/36)
abs_balance = p.abs(balance)
# %time ans = p.minimize(abs_balance) # fast - 0.2 s
%time ans = p.minimize(abs_balance - 1e-3*tot_thrust) # slower - 1.3 s

x = []
for eng in N_eng:
    cnt = round(ans['by_var'][eng])
    x.append(cnt)
    if cnt > 0:
        print(f'{cnt:.0f}    {eng.name}')
print(f'Total cost:   {np.dot(engines.weight, x)} of {max_cost}')
print(f'Total thrust: {np.dot(engines.thrust, x)}')
print(f'Total turn:   {np.dot(engines.turn, x)/36}')
# print(f'Objective:    {as_expr(abs_balance).eval(ans['by_var'])}')
print(f'Objective:    {ans['by_var'][abs_balance]}')
# print(f'Objective:    {ans['soln'].fun}')

## Balance thrust and turning with min()

This increases both values because they don't have to be exactly equal!

In [ ]:
p = limp.Problem()
max_cost = 210 # weight limit our ship can carry

N_eng = [p.intvar(row.name, 0, max_cost // row.weight) for row in engines.itertuples()]
p.constraint( p.sum(N_eng, engines.weight), max_cost )
tot_turn = p.contvar('total_turning', 0, 1e6)
p.equal(tot_turn, p.sum(N_eng, engines.turn/36))
tot_thrust = p.contvar('total_thrust', 0, 1e6)
p.equal(tot_thrust, p.sum(N_eng, engines.thrust))
balance = p.min([tot_turn, tot_thrust])
%time ans = p.maximize(balance)

print(ans['soln'].message)
x = []
for eng in N_eng:
    cnt = round(ans['by_var'][eng])
    x.append(cnt)
    if cnt > 0:
        print(f'{cnt:.0f}    {eng.name}')
print(f'Total cost:   {np.dot(engines.weight, x)} of {max_cost}')
print(f'Total thrust: {np.dot(engines.thrust, x)}')
print(f'Total turn:   {np.dot(engines.turn, x)/36}')
# print(f'Objective:    {as_expr(abs_balance).eval(ans['by_var'])}')
print(f'Objective:    {ans['soln'].fun}')